命名实体识别(NER)是根据类别对标记进行分类的任务，例如将标记标识为个人、组织或位置。命名实体识别数据集的一个例子是CoNLL-2003数据集，它完全基于该任务。

In [1]:
!pip install transformers
import torch

     |████████████████████████████████| 573kB 41.1MB/s 
     |████████████████████████████████| 870kB 50.8MB/s 
     |████████████████████████████████| 1.0MB 40.7MB/s 
     |████████████████████████████████| 3.7MB 49.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=fd82f82189f8189e575a64cee6267fab7ef312aaf8088e3a193a24ed5cf4aa00
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
torch.cuda.get_device_name(0)

'Tesla P4'

### 下面是一个使用模型和Tokenizer进行命名实体识别的示例。该过程如下：

– 从checkpoint名称实例化一个tokenizer和一个模型。该模型被识别为一个BERT模型，并用存储在checkpoint中的权重加载它。

– 定义用于训练模型的标签列表。

– 定义一个包含已知实体的序列，例如“Hugging Face”作为一个组织，“New York City”作为一个位置。

– 将单词拆分为标记，以便它们可以映射到预测。使用一个小技巧，首先对序列进行完全的编码和解码，这样就留下了一个包含特殊标记的字符串。

– 将该序列编码为id(自动添加特殊标记)。

– 通过将输入传递到模型并获得第一个输出来检索预测。这将导致每个标记在9个可能的类上分布。我们使用argmax来检索每个标记最可能的类。

– 将每个标记及其预测到一起并打印出来。

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # 不是命名实体
    "B-MISC",  # 一个杂项实体的开头
    "I-MISC",  # 杂项
    "B-PER",   # 一个人名的开头
    "I-PER",   # 人名
    "B-ORG",   # 一个组织的开头
    "I-ORG",   # 组织
    "B-LOC",   # 一个地点的开头
    "I-LOC"    # 地点
]

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" \
           "close to the Manhattan Bridge."



In [4]:
# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

[('[CLS]', 'O'), ('Hu', 'I-ORG'), ('##gging', 'I-ORG'), ('Face', 'I-ORG'), ('Inc', 'I-ORG'), ('.', 'O'), ('is', 'O'), ('a', 'O'), ('company', 'O'), ('based', 'O'), ('in', 'O'), ('New', 'I-LOC'), ('York', 'I-LOC'), ('City', 'I-LOC'), ('.', 'O'), ('Its', 'O'), ('headquarters', 'O'), ('are', 'O'), ('in', 'O'), ('D', 'I-LOC'), ('##UM', 'I-LOC'), ('##BO', 'I-LOC'), (',', 'O'), ('therefore', 'O'), ('very', 'O'), ('##c', 'O'), ('##lose', 'O'), ('to', 'O'), ('the', 'O'), ('Manhattan', 'I-LOC'), ('Bridge', 'I-LOC'), ('.', 'O'), ('[SEP]', 'O')]


这将输出映射到其预测的每个标记的列表。与管道不同的是，这里每个标记都有一个预测，因为我们没有删除“O”类，这意味着在该标记上找不到特定的实体。